In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("chinook.db")

In [97]:
emp = pd.read_sql_query("select * from employees", conn)
emp

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
5,6,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
6,7,King,Robert,IT Staff,6.0,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
7,8,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [8]:
track = pd.read_sql_query("select * from tracks where TrackId = 4", conn)
track

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99


In [15]:
genre = pd.read_sql_query("select trk.Name as TrackNm, trk.Composer, trk.UnitPrice, gnr.Name as GenreNm, gnr.GenreId, trk.TrackId \
                   from tracks as trk, genres as gnr \
                   where trk.GenreId = gnr.GenreId and trk.TrackId = 4 and gnr.GenreId = 1", conn)
genre.head()

,TrackNm,Composer,UnitPrice,GenreNm,GenreId,TrackId
0,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",0.99,Rock,1,4


In [22]:
pd.read_sql_query("SELECT * FROM media_types", conn)

,MediaTypeId,Name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


In [87]:
ttl_sales = pd.read_sql_query("SELECT inv.BillingCountry as Country, gnr.Name as Genre, inv.InvoiceDate, SUM(inv.Total) as Total, AVG(inv.Total) as rataan \
                   FROM invoices as inv, invoice_items as inv_itm, tracks as trk, genres as gnr, media_types as md_typ \
                   WHERE \
                   inv.InvoiceId = inv_itm.InvoiceId \
                   AND \
                   inv_itm.TrackId = trk.TrackId \
                   AND \
                   trk.GenreId = gnr.GenreId \
                   AND \
                   trk.MediaTypeId = md_typ.MediaTypeId \
                   AND \
                   md_typ.MediaTypeId = 3 \
                   GROUP BY Genre \
                   ORDER BY Total", conn, parse_dates='InvoiceDate')

ttl_sales['Month'] = ttl_sales["InvoiceDate"].dt.month_name()
ttl_sales

,Country,Genre,InvoiceDate,Total,rataan,Month
0,Chile,Science Fiction,2010-01-13,102.41,17.068333,January
1,USA,Comedy,2012-09-27,112.30,12.477778,September
2,USA,Sci Fi & Fantasy,2012-08-05,198.87,9.943500,August
3,Czech Republic,Drama,2013-11-13,544.61,18.779655,November
4,Sweden,TV Shows,2010-01-10,817.71,17.398085,January


In [55]:
ttl_sales['month_year'] = pd.to_datetime(ttl_sales['InvoiceDate']).dt.to_period('M')
ttl_sales

,Country,Genre,InvoiceDate,Total,Month,month_year
0,Chile,Science Fiction,2010-01-13,102.41,January,2010-01
1,USA,Comedy,2012-09-27,112.30,September,2012-09
2,USA,Sci Fi & Fantasy,2012-08-05,198.87,August,2012-08
3,Czech Republic,Drama,2013-11-13,544.61,November,2013-11
4,Sweden,TV Shows,2010-01-10,817.71,January,2010-01


In [64]:
ttl_sales['month_year'] = ttl_sales['InvoiceDate'].dt.strftime('%m-%Y')
ttl_sales

,Country,Genre,InvoiceDate,Total,Month,month_year
0,Chile,Science Fiction,2010-01-13,102.41,January,01-2010
1,USA,Comedy,2012-09-27,112.30,September,09-2012
2,USA,Sci Fi & Fantasy,2012-08-05,198.87,August,08-2012
3,Czech Republic,Drama,2013-11-13,544.61,November,11-2013
4,Sweden,TV Shows,2010-01-10,817.71,January,01-2010


In [32]:
country_sales = pd.read_sql_query("SELECT inv.BillingCountry as Country, gnr.Name as Genre, md_typ.Name as Media_Nm, sum(inv.Total) as Total\
                   FROM invoices as inv, invoice_items as inv_itm, tracks as trk, genres as gnr, media_types as md_typ \
                   WHERE \
                   inv.InvoiceId = inv_itm.InvoiceId \
                   AND \
                   inv_itm.TrackId = trk.TrackId \
                   AND \
                   trk.GenreId = gnr.GenreId \
                   AND \
                   trk.MediaTypeId = md_typ.MediaTypeId \
                   AND \
                   inv.BillingCountry = 'Germany' \
                   GROUP BY Genre \
                   ORDER BY Total DESC", conn)
country_sales

,Country,Genre,Media_Nm,Total
0,Germany,Rock,Protected AAC audio file,577.17
1,Germany,Metal,MPEG audio file,189.09
2,Germany,Latin,MPEG audio file,163.35
3,Germany,Alternative & Punk,MPEG audio file,105.12
4,Germany,Blues,MPEG audio file,97.02
5,Germany,Soundtrack,MPEG audio file,59.40
6,Germany,TV Shows,Protected MPEG-4 video file,44.73
7,Germany,Heavy Metal,MPEG audio file,41.58
8,Germany,Science Fiction,Protected MPEG-4 video file,29.82
9,Germany,Jazz,MPEG audio file,27.72


In [46]:
ttl_sales_media = pd.read_sql_query("SELECT inv.BillingCountry as Country, gnr.Name as Genre, md_typ.Name, SUM(inv.Total) as Total \
                   FROM invoices as inv, invoice_items as inv_itm, tracks as trk, genres as gnr, media_types as md_typ \
                   WHERE \
                   inv.InvoiceId = inv_itm.InvoiceId \
                   AND \
                   inv_itm.TrackId = trk.TrackId \
                   AND \
                   trk.GenreId = gnr.GenreId \
                   AND \
                   trk.MediaTypeId = md_typ.MediaTypeId \
                   AND \
                   md_typ.MediaTypeId = 3 \
                   GROUP BY Genre \
                   ORDER BY Total", conn)
ttl_sales_media

,Country,Genre,Name,Total
0,Chile,Science Fiction,Protected MPEG-4 video file,102.41
1,USA,Comedy,Protected MPEG-4 video file,112.30
2,USA,Sci Fi & Fantasy,Protected MPEG-4 video file,198.87
3,Czech Republic,Drama,Protected MPEG-4 video file,544.61
4,Sweden,TV Shows,Protected MPEG-4 video file,817.71


In [67]:
album = pd.read_sql_query("select * from albums", conn)
album.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [86]:
total_sls = pd.read_sql_query("SELECT abm.Title as Album, art.Name as Artist, sum(inv.Total) as TotalSales\
                   FROM \
                   invoices as inv, invoice_items as inv_itm, tracks as trk, albums as abm, artists as art \
                   WHERE \
                   inv.InvoiceId = inv_itm.InvoiceId \
                   AND \
                   inv_itm.TrackId = trk.TrackId \
                   AND \
                   trk.AlbumId = abm.AlbumId \
                   AND \
                   abm.ArtistId = art.ArtistId \
                   GROUP BY abm.Title \
                   ORDER BY TotalSales DESC", conn)
total_sls.head(10)

,Album,Artist,TotalSales
0,Greatest Hits,Lenny Kravitz,372.51
1,"Lost, Season 2",Lost,290.18
2,"Heroes, Season 1",Heroes,238.61
3,"Lost, Season 1",Lost,223.65
4,"Lost, Season 3",Lost,211.80
5,"Battlestar Galactica, Season 3",Battlestar Galactica,202.80
6,Minha Historia,Chico Buarque,185.13
7,"The Office, Season 3",The Office,170.93
8,Ao Vivo [IMPORT],Zeca Pagodinho,161.74
9,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),157.10


In [ ]:
ttl_sales = pd.read_sql_query("")
ttl_sales.head()

In [115]:
emp_sales = pd.read_sql_query("SELECT (emp.FirstName||' '||emp.LastName) as FullName, \
                                      sum(inv.Total) as TotalSales, inv.InvoiceDate as Period \
                         FROM \
                         employees as emp, customers as cst, invoices as inv \
                         WHERE \
                         emp.EmployeeId = cst.SupportRepId \
                         AND \
                         cst.CustomerId = inv.CustomerId \
                         AND \
                         emp.Title = 'Sales Support Agent' \
                         GROUP BY FullName, inv.InvoiceDate", conn, parse_dates='Period')

emp_sales['Period'] = pd.to_datetime(emp_sales['Period']).dt.to_period('M')                         
emp_sales.head()

,FullName,TotalSales,Period
0,Jane Peacock,0.99,2009-01
1,Jane Peacock,1.98,2009-02
2,Jane Peacock,3.96,2009-02
3,Jane Peacock,5.94,2009-02
4,Jane Peacock,8.91,2009-02


In [120]:
emp_sales_nm = emp_sales.pivot_table(
    index= "Period",
    values= "TotalSales",
    aggfunc='sum'
).sort_values(by="Period", ascending=True)

emp_sales_nm.head()

,TotalSales
Period,
2009-01,35.64
2009-02,37.62
2009-03,37.62
2009-04,37.62
2009-05,37.62


In [122]:
pd.crosstab(
    index=emp_sales['Period'], 
    columns=emp_sales['FullName'],
    values=emp_sales['TotalSales'],
    aggfunc = 'sum')

FullName,Jane Peacock,Margaret Park,Steve Johnson
Period,,,
2009-01,0.99,23.76,10.89
2009-02,20.79,2.97,13.86
2009-03,1.98,13.86,21.78
2009-04,18.81,16.83,1.98
2009-05,10.89,1.98,24.75
2009-06,1.98,10.89,24.75
2009-07,22.77,3.96,10.89
2009-08,30.69,6.93,NaN
2009-09,0.99,28.71,7.92
